In [98]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

from nltk import word_tokenize 
from nltk.util import ngrams
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer

In [7]:
# read in the HT data file
data_file = "../Data/ComplexDataLab_Canadian_OctoberBatch.csv"
data = pd.read_csv(data_file, index_col=0)
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PostingDate,Url,l_Location,u_Age,u_Description,u_Title,u_Location,e_PhoneNumbers,u_PhoneNumbers,e_Emails
0,Thu 14 Apr 2016 11:40 AM,https://independent.com.calgary.listcrawler.co...,calgary,31,Take the time to enjoy your night with me.. 40...,Sinfully sweet,"Calgary, Downtown, Mins from Downtown, Northea...",['4039715391'],['4039715391'],[]
1,Tue 04 Jul 2017 08:30 AM,https://yolo.com.calgary.listcrawler.com/post/...,calgary,28,Maria Vega Latina for Real!!!Plz read my rate...,Tired fake AddsFake Expectations36 DDD Real 10...,"Calgary, Downtown, DOWNTOWN CORE INCALL and OU...",['4389267478'],['4389267478'],[]
2,Mon 03 Jul 2017 07:04 AM,https://milfy.com.toronto.listcrawler.com/post...,toronto,34,Sexual Deviant LOVES 2ROCK IT NAUGHTY! My sexu...,New Halton Girl! Outcall MIND BL0-WING DEVIANT...,"Burlington, Hamilton, Oakville milton, Oakvill...",['2897801427'],['2897801427'],[]
3,Thu 29 Sep 2016 12:06 AM,https://max80.com.vancouver.listcrawler.com/po...,vancouver,25,778 8four6 4234 Hi! I'm Tiffany baby! ;) LETS ...,Early Morning Drop In! Rates $40-$100 TEXT NOW!,"City of Vancouver, MARINE &amp; CAMBIE, Vancouver",[],['7788464234'],[]
4,Tue 03 May 2016 06:16 PM,https://milfy.com.barrie.listcrawler.com/post/...,barrie,33,HI GUYS ! I CANT WAIT TO MEET YOU ! I AM 33 5...,"BARBIE aka (Cheyenne) 33 fetishes welcome, soa...","Barrie, Barrie and surrounding areas",['7053314617'],['7053314617'],[]


In [8]:
# size of the data
data.shape

(932888, 10)

In [13]:
data = data.drop_duplicates()
data.shape

(932888, 10)

In [20]:
# randomly sample some data points 
sampled_data = data.sample(n=1000)

In [21]:
sampled_data.head()

,PostingDate,Url,l_Location,u_Age,u_Description,u_Title,u_Location,e_PhoneNumbers,u_PhoneNumbers,e_Emails
127401,Wed 07 Jun 2017 08:47 AM,https://independent.com.calgary.listcrawler.co...,calgary,26,I Woke up feeling extra frisky and FUN! Allow ...,"Start your morning off right breathtaking, gre...","Calgary, Downtown, Incall Downtown 7Ave/7St SW...",['4032067785'],['4032067785'],[]
910548,Tue 10 May 2016 03:03 PM,https://hongkongbobo.com.toronto.listcrawler.c...,toronto,21,AMIKO & nana < Hi!!!! We are new in town . We...,..OUT Call 24/7 HOTTEST ASIAN PARTY Girls IN ...,"Brampton, Durham Region, Toronto, Outcall Anyw...",['6476404510'],['6476404510'],['647-640-4510Outcall@Toronto']
912484,Tue 25 Jun 2019 12:02 PM,https://yolo.com.calgary.listcrawler.com/post/...,calgary,28,Text me and let hookup to have a great fun .....,😧super honry seeking hard fucker,Canada,['2168154471'],['2168154471'],[]
583604,Thu 18 Jan 2018 04:15 PM,https://luxerotica.com.kingston.listcrawler.co...,kingston,25,New in town you need the best companionship wi...,EvA 819-919-6459,"Kingston, Kingston motel super",['8199196459'],['8199196459'],[]
148399,Fri 24 Jun 2016 10:36 AM,https://yolo.com.calgary.listcrawler.com/post/...,calgary,22,"Hi Guys im COCO *OPEN MIND-PARTY GIRL* Sexy, ...",NAUGHTY !!IN& OUT1000% real! GORGEOUS -- SEXY...,"Calgary, Calgary Incall or Outcall, Downtown","['1587718122', '5877181222']",['5877181222'],[]


In [94]:
data['label'] = None
data['group'] = None
grp_id = 1
for grp in data.groupby('u_PhoneNumbers'):
    ids = grp[1].index.values
    if len(ids) <= 1:
        data['label'].iloc[ids] = 0
    else:
        data['label'].iloc[ids] = 1
    data['group'].iloc[ids] = grp_id
    grp_id += 1

In [95]:
data.head()

,PostingDate,Url,l_Location,u_Age,u_Description,u_Title,u_Location,e_PhoneNumbers,u_PhoneNumbers,e_Emails,label,group
0,Thu 14 Apr 2016 11:40 AM,https://independent.com.calgary.listcrawler.co...,calgary,31,Take the time to enjoy your night with me.. 40...,Sinfully sweet,"Calgary, Downtown, Mins from Downtown, Northea...",['4039715391'],['4039715391'],[],1,23710
1,Tue 04 Jul 2017 08:30 AM,https://yolo.com.calgary.listcrawler.com/post/...,calgary,28,Maria Vega Latina for Real!!!Plz read my rate...,Tired fake AddsFake Expectations36 DDD Real 10...,"Calgary, Downtown, DOWNTOWN CORE INCALL and OU...",['4389267478'],['4389267478'],[],1,31764
2,Mon 03 Jul 2017 07:04 AM,https://milfy.com.toronto.listcrawler.com/post...,toronto,34,Sexual Deviant LOVES 2ROCK IT NAUGHTY! My sexu...,New Halton Girl! Outcall MIND BL0-WING DEVIANT...,"Burlington, Hamilton, Oakville milton, Oakvill...",['2897801427'],['2897801427'],[],1,16253
3,Thu 29 Sep 2016 12:06 AM,https://max80.com.vancouver.listcrawler.com/po...,vancouver,25,778 8four6 4234 Hi! I'm Tiffany baby! ;) LETS ...,Early Morning Drop In! Rates $40-$100 TEXT NOW!,"City of Vancouver, MARINE &amp; CAMBIE, Vancouver",[],['7788464234'],[],1,80398
4,Tue 03 May 2016 06:16 PM,https://milfy.com.barrie.listcrawler.com/post/...,barrie,33,HI GUYS ! I CANT WAIT TO MEET YOU ! I AM 33 5...,"BARBIE aka (Cheyenne) 33 fetishes welcome, soa...","Barrie, Barrie and surrounding areas",['7053314617'],['7053314617'],[],1,75746


In [49]:
def get_ngrams(text):
    tokens = word_tokenize(text)
    tokensProcessed = [token.lower() for token in tokens if token not in stopwords.words('english')]
    bigrams = list(ngrams(tokensProcessed, 2))
    unigrams = list(ngrams(tokensProcessed, 1))
    return unigrams, bigrams

In [126]:
def get_loc_features(text):
    vectorizer = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1,1))
    features = vectorizer.fit_transform(text)
    return features.toarray(), vectorizer.get_feature_names()

In [124]:
def get_features(text):
    vectorizer_uni = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1,1))
    features_uni = vectorizer_uni.fit_transform(text)
    vectorizer_bi = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(2,2))
    features_bi = vectorizer_bi.fit_transform(text)
    return features_uni.toarray(), vectorizer_uni.get_feature_names(), features_bi.toarray(), vectorizer_bi.get_feature_names()

In [125]:
desc_features_uni, desc_feature_names_uni, desc_features_bi, desc_feature_names_bi = get_features(sampled_data.u_Description)
title_features_uni, title_feature_names_uni, title_features_bi, title_feature_names_bi = get_features(sampled_data.u_Title)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [127]:
loc_features, loc_feature_names = get_loc_features(sampled_data.u_Location)

In [129]:
title_features.shape

(1000, 9964)

In [131]:
feature_vectors = np.concatenate((desc_features_uni, desc_features_bi, title_features_uni, title_features_bi, title_features_bi), axis=1)

In [132]:
feature_vectors.shape

(1000, 49936)

In [133]:
sampled_data_labels = data.iloc[sampled_data.index].label.values

In [134]:
pkl.dump(sampled_data_labels, open("sampled_data_labels.pkl","wb"))
pkl.dump(feature_vectors, open("sampled_data_features.pkl","wb"))

In [135]:
feature_names = np.append(np.append(np.append(np.append(desc_feature_names_uni, desc_feature_names_bi), title_feature_names_uni), title_feature_names_bi), loc_feature_names).shape

In [136]:
pkl.dump(feature_names, open("sample_data/sampled_data_feature_names.pkl","wb"))

In [137]:
pkl.dump(desc_feature_names_uni, open("sample_data/desc_feature_names_uni.pkl","wb"))
pkl.dump(desc_feature_names_bi, open("sample_data/desc_feature_names_bi.pkl","wb"))
pkl.dump(title_feature_names_uni, open("sample_data/title_feature_names_uni.pkl","wb"))
pkl.dump(title_feature_names_bi, open("sample_data/title_feature_names_bi.pkl","wb"))
pkl.dump(loc_feature_names, open("sample_data/loc_feature_names.pkl","wb"))